You can access the accompanying video here: 

<a href="https://blinks.bloomberg.com/screens/PLYR%20VOD%20361030021"> Animations (8 minutes)</a>

In [1]:
import ipywidgets as widgets
import bqplot as bqp
import pandas as pd
import numpy as np 
import bql 
bq = bql.Service()

In [2]:
# Create the data for the line chart

px = bq.func.dropna(bq.data.px_last(dates=bq.func.range('-3m', '0d')))/10

index = 'SPX Index'

px_bql_request = bql.Request(index, px)
px_bql_response = bq.execute(px_bql_request)
px_df = bql.combined_df(px_bql_response).reset_index()

In [3]:
# Create the data for the barchart
members = bq.univ.members(index)
pct_diff = px.pct_diff()
match = bq.func.matches(pct_diff,pct_diff<0).dropna(True)

count_group = bq.func.group(match,match['DATE']).count()
request = bql.Request(members,count_group)
resp = bq.execute(request)

count_df = bql.combined_df(resp).reset_index()

In [4]:
# Create figure for animation 
def create_figure():
    # Create scales
    scale_x = bqp.LinearScale(min=0,max=60)
    scale_y = bqp.LinearScale(min=0,max=500)

    # Create mark
    tooltip = bqp.Tooltip(fields=['y'], show_labels=False, formats=['.3f'])
    mark_bar = bqp.Bars(x=[],
                        y=[],
                        scales={'x': scale_x, 'y': scale_y},
                        padding=0.1,
                        colors=['#1B84ED'],
                        tooltip=bqp.Tooltip(fields=['y'],
                                            show_labels=False,
                                            formats=['.3f']))

    mark_line = bqp.Lines(x=[],
                          y=[],
                          stroke_width=3,
                          scales={'x': scale_x, 'y': scale_y},
                          colors=['#CF7DFF', '#FF5A00', '#00D3D6',
                                  '#008616', '#B31D83', '#FF1E3E', '#FF9E24',
                                  '#63B2FF', '#30C030'])

    # Create Axes
    axis_y = bqp.Axis(scale=scale_y,
                      orientation='vertical',
                      label='Y Label')
    axis_x = bqp.Axis(scale=scale_x,
                      tick_rotate=20,
                      tick_style={'text-anchor': 'start'})

    # Create Figure
    figure = bqp.Figure(marks=[mark_bar,mark_line],
                        axes=[axis_x, axis_y],
                        title='Vertical bar plot',
                        title_style={'font-size': '22px'},
                        padding_y=0,
                        fig_margin={'top': 50, 'bottom': 60,
                                    'left': 60, 'right': 50},
                        layout={'width': '100%',
                                'height': '500px'})
    return figure

# Define the update plot function for the animation
def update_plot(figure):
    barplot = figure.marks[0]
    barplot.x = list(count_df.iloc[0:slider.value].index)
    barplot.y = list(count_df.iloc[0:slider.value][count_df.iloc[0:slider.value].columns[-1]])
    
    lineplot = figure.marks[1]
    lineplot.x = list(px_df.iloc[0:slider.value].index)
    lineplot.y = list(px_df.iloc[0:slider.value][px_df.iloc[0:slider.value].columns[-1]])


In [5]:
# Define the callback function for the animation slider
def slider_change_callback(arg1):
    if arg1['name']!='value':
        return update_plot(figure)

In [6]:
figure = create_figure()
# Create the animation widget
play = widgets.Play(
#     interval=10
    value=0,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))

# Observe the callback function when the slider changes value
slider.observe(slider_change_callback)

# Output the final dashboard
widgets.VBox([widgets.HBox([play, slider]),figure])